# 🌾 AgriChat (TinyLlama Fallback Version)

**Lightweight edition — runs instantly on free Colab GPU (T4)**

✅ Automatically loads the base TinyLlama 1.1B Chat model
✅ Falls back gracefully if LoRA adapter is missing
✅ Short, clear answers (1–2 sentences)
✅ Keeps chat history visible
✅ Includes Download Chat History button

---

## 1️⃣ Install dependencies

In [ ]:
!pip install -q transformers peft bitsandbytes safetensors sentencepiece gradio accelerate

## 2️⃣ Launch AgriChat (Fallback Chat Mode)
This version skips training and starts chat instantly.

In [ ]:

import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch, os

base = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
adapter_path = "./outputs/lora_adapter"

def load_model():
    print("🔍 Loading base TinyLlama model...")
    model = AutoModelForCausalLM.from_pretrained(base, device_map="auto", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(base)
    if os.path.exists(adapter_path):
        print("✅ Found LoRA adapter — loading fine-tuned weights...")
        model = PeftModel.from_pretrained(model, adapter_path)
    else:
        print("⚠️ No LoRA adapter found — using base TinyLlama model only.")
    model.eval()
    return tokenizer, model

tokenizer, model = load_model()

def generate_reply(prompt, history):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=128).to('cuda')
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=60, do_sample=False, temperature=0.3)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    sentences = text.split('.')
    short = '.'.join([s.strip() for s in sentences[:2] if s.strip()])
    if short and not short.endswith('.'):
        short += '.'
    history.append((prompt, short))
    return history, history

# --- Gradio Chat UI ---
with gr.Blocks(title="AgriChat — TinyLlama Fallback") as demo:
    gr.Markdown("# 🌾 AgriChat — Your Smart Agriculture Assistant (TinyLlama)")
    gr.Markdown("Ask agriculture-related or customer-service style questions. Short and clear answers (1–2 sentences).")
    chatbot = gr.Chatbot(label="AgriChat Conversation", height=400)
    msg = gr.Textbox(label="Ask something...", placeholder="Type your question here")
    clear = gr.Button("Clear Chat")
    save = gr.Button("💾 Download Chat History")

    def save_chat(history):
        text = "\n".join([f"You: {h[0]}\nAgriChat: {h[1]}" for h in history])
        path = "/content/agri_chat_history.txt"
        with open(path, "w") as f:
            f.write(text)
        return path

    msg.submit(generate_reply, [msg, chatbot], [chatbot, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)
    save.click(save_chat, [chatbot], gr.File(label="Download your chat"))

demo.launch(share=False, inbrowser=False)


✅ **Instructions:**
1. Go to Runtime → Change runtime type → **GPU** → Save.
2. Run all cells from top to bottom.
3. The chatbot will open below.
4. Type your question (e.g., *What is organic farming?*) and press Enter.
5. Use 💾 Download Chat History to save your conversation.